4. Tenemos dos grafos no dirigidos $G_1$ y $G_2$ , con la misma cantidad de vértices y aristas. $G_1$ es un grafo aleatorio de Erdös-Rényi, mientras $G_2$ es un grafo que cumple la ley de potencias en la distribución de los grados. Consideremos un virus que comienza en un único vértice aleatorio y se expande según el modelo **SIR**.

    a. ¿En cuál grafo es más probable que ocurra una epidemia (i.e. se infecte al menos un 30% de la red)? Justificar brevemente la respuesta. 

    b. Supongamos que en vez de comenzar en un vértice aleatorio, la epidemia comenzara en el vértice de mayor grado de $G_1$ y $G_2$ , respectivamente. ¿En cuál de los grafos es más probable que ocurra una epidemia? Justificar brevemente la respuesta.

    c. ¿Cómo afecta la existencia (o no existencia) de comunidades en la expansión de la epidemia?

    Para responder estas preguntas, se les recomienda realizar simulaciones. Pueden agregar todo tipo de resultados obtenidos para justificar sus respuestas.

**[3 Puntos]**

En el modelo de Erdös-Rényi, el coeficiente de clustering es muy bajo, con lo que 

In [121]:
import networkx as nx
import ndlib.models.epidemics as ep
import ndlib.models.ModelConfig as mc
import sys
sys.path.append('../')
from social_networks_utils.modelos import preferential_attachment, erdos_renyi
from social_networks_utils.metricas import grado_promedio
import pandas as pd

In [118]:
g1 = preferential_attachment(False, 2.5, 1000, 10)

In [119]:
len(g1.edges()), len(g1.nodes())

(13256, 1000)

In [129]:
g2 = erdos_renyi(1000, grado_promedio(g1))

In [131]:
len(g2.edges()), len(g2.nodes())

(13244, 1000)

In [132]:
model1 = ep.SIRModel(g1)
model2 = ep.SIRModel(g2)

In [133]:
# Model Configuration
cfg = mc.Configuration()
cfg.add_model_parameter('beta', 0.01)
cfg.add_model_parameter('gamma', 0.005)
cfg.add_model_parameter("fraction_infected", 0.05)

In [134]:
model1.set_initial_status(cfg)
model2.set_initial_status(cfg)

In [135]:
def infeccion_alcanzada(iteracion, porcentaje):
    total_nodes = iteracion["node_count"][0]+iteracion["node_count"][1]+iteracion["node_count"][2]
    return (iteracion["node_count"][1]+iteracion["node_count"][2])/total_nodes >= (porcentaje/100)

def epidemia_recuperada(iteracion):
    return iteracion["node_count"][1] == 0 

In [145]:
veces_infeccion_alcanzada = {}

In [146]:

model1.set_initial_status(cfg)
model2.set_initial_status(cfg)

for _ in range(4000):
    model1.reset()
    model2.reset()
    
    iteracion1 = model1.iteration()
    iteracion2 = model2.iteration()
    
    while((not infeccion_alcanzada(iteracion1, 30) and not infeccion_alcanzada(iteracion2, 30))):
        iteracion1 = model1.iteration()
        iteracion2 = model2.iteration()
    
    if(infeccion_alcanzada(iteracion1, 30)):
        veces_infeccion_alcanzada["model1"] = veces_infeccion_alcanzada.get("model1", 0) + 1
    if(infeccion_alcanzada(iteracion2, 30)):
        veces_infeccion_alcanzada["model2"] = veces_infeccion_alcanzada.get("model2", 0) + 1


In [147]:
veces_infeccion_alcanzada

{'model1': 2865, 'model2': 2811}